# Step 3. 두 개의 KB 간 내용 비교

### Setup
* 아래 패키지 설치 후, 커널 Restart 해주세요.

In [ ]:
%pip install --upgrade pip
%pip install boto3 --force-reinstall --quiet
%pip install botocore --force-reinstall --quiet
%pip install sqlalchemy==2.0.0 --quiet
%pip install langchain --force-reinstall --quiet
%pip install langchain_aws langchain-community --force-reinstall --quiet

### Bedrock Client 초기화

In [1]:
import boto3
import pprint
from botocore.client import Config
import json
import time
import os

pp = pprint.PrettyPrinter(indent=2)
session = boto3.session.Session()
region = session.region_name
bedrock_config = Config(connect_timeout=120, read_timeout=120, retries={'max_attempts': 0})
bedrock_client = boto3.client('bedrock-runtime', region_name = region)
bedrock_agent_client = boto3.client("bedrock-agent-runtime",
                              config=bedrock_config, region_name = region)
print(region)

us-east-1


### LangChain 초기화

In [2]:
import langchain
from langchain_aws import ChatBedrock
from langchain.retrievers.bedrock import AmazonKnowledgeBasesRetriever


modelId = 'anthropic.claude-3-sonnet-20240229-v1:0' # change this to use a different version from the model provider

llm = ChatBedrock(model_id=modelId, 
                  client=bedrock_client)

### 사전에 생성한 KB 목록 확인
* 사용하려는 2개의 KB ID를 확인 합니다.

In [3]:
import boto3

def list_knowledge_bases():
    # Bedrock 클라이언트 생성
    bedrock_agent = boto3.client('bedrock-agent')

    # Knowledge Base 목록 가져오기
    response = bedrock_agent.list_knowledge_bases(
        maxResults=100  # 한 번에 가져올 최대 결과 수
    )

    # 결과 출력
    if 'knowledgeBaseSummaries' in response:
        print("Found Knowledge Bases:")
        for kb in response['knowledgeBaseSummaries']:
            print(f"- Name: {kb['name']}")
            print(f"  ID: {kb['knowledgeBaseId']}")
            print(f"  Description: {kb.get('description', 'N/A')}")
            print(f"  Status: {kb['status']}")
            print(f"  Last Updated: {kb['updatedAt']}")
            print("---")
    else:
        print("No Knowledge Bases found.")

    # 페이지네이션 처리
    while 'nextToken' in response:
        response = bedrock_agent.list_knowledge_bases(
            maxResults=100,
            nextToken=response['nextToken']
        )
        
        for kb in response['knowledgeBaseSummaries']:
            print(f"- Name: {kb['name']}")
            print(f"  ID: {kb['knowledgeBaseId']}")
            print(f"  Description: {kb.get('description', 'N/A')}")
            print(f"  Status: {kb['status']}")
            print(f"  Last Updated: {kb['updatedAt']}")
            print("---")

if __name__ == "__main__":
    list_knowledge_bases()

Found Knowledge Bases:
- Name: 2-civil-eng-01
  ID: VIIQSV2GCL
  Description: N/A
  Status: ACTIVE
  Last Updated: 2024-07-23 03:06:15.586610+00:00
---
- Name: 240722-test-kb
  ID: PIWYW746HW
  Description: N/A
  Status: ACTIVE
  Last Updated: 2024-07-22 04:51:26.268276+00:00
---
- Name: 2-civil-eng-02
  ID: LO9DRKGBTX
  Description: N/A
  Status: ACTIVE
  Last Updated: 2024-07-23 03:08:42.294328+00:00
---
- Name: knowledge-base-quick-start-aafi3
  ID: KRQKUHNJIV
  Description: N/A
  Status: ACTIVE
  Last Updated: 2024-02-26 08:27:55.881890+00:00
---
- Name: knowledge-base-quick-start-en
  ID: DJLWVWNGBT
  Description: N/A
  Status: ACTIVE
  Last Updated: 2024-02-26 12:26:22.625358+00:00
---
- Name: 1-es-eng
  ID: 2S7W5D8WOO
  Description: N/A
  Status: ACTIVE
  Last Updated: 2024-07-23 02:55:14.020228+00:00
---


In [4]:
# 비교에 사용할 2개의 KB ID 를 확인하고 수정

kb_id_1 = "VIIQSV2GCL"  # 2-civil-eng-01
kb_id_2 = "LO9DRKGBTX"  # 2-civil-eng-02

In [5]:
# 각각의 kb_id 에 대한 Name 확인

def list_knowledge_bases(kb_id):
    bedrock_agent = boto3.client('bedrock-agent')
    target_id = kb_id
    target_name = None

    response = bedrock_agent.list_knowledge_bases(maxResults=100)

    while True:
        for kb in response.get('knowledgeBaseSummaries', []):
            if kb['knowledgeBaseId'] == target_id:
                target_name = kb['name']
                break

        if target_name or 'nextToken' not in response:
            break

        response = bedrock_agent.list_knowledge_bases(
            maxResults=100,
            nextToken=response['nextToken']
        )

    if target_name:
        print(f"Knowledge Base with ID {target_id} is named: {target_name}")
        return target_name
    else:
        print(f"No Knowledge Base found with ID {target_id}")


# KB Name 확인
kb_name_1 = list_knowledge_bases(kb_id_1)
kb_name_2 = list_knowledge_bases(kb_id_2)

Knowledge Base with ID VIIQSV2GCL is named: 2-civil-eng-01
Knowledge Base with ID LO9DRKGBTX is named: 2-civil-eng-02


In [6]:
from langchain.prompts import PromptTemplate

# 프롬프트 템플릿
PROMPT_TEMPLATE = """
Human: You are a ITB advisor AI system, and provides answers to questions by using fact based when possible. 
Use the following pieces of information to provide a detail answer to the question enclosed in <question> tags. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.

<context>
{context}
</context>


<question>
{question}
</question>

The response should be specific and use statistics or numbers when possible.

Assistant:"""
claude_prompt = PromptTemplate(template=PROMPT_TEMPLATE, 
                               input_variables=["context","question"])



# 두 KB에 대해서 공통으로 물어볼 사용자 쿼리
query = """
이 문서의 전체 내용에 대해서 자세하게 정리해주세요.
"""

In [7]:
%%time

# KB에서 관련 내용 검색
retriever1 = AmazonKnowledgeBasesRetriever(
        knowledge_base_id=kb_id_1,
        retrieval_config={"vectorSearchConfiguration": 
                          {"numberOfResults": 10,
                           'overrideSearchType': "HYBRID", # optional
                           }
                          },
        # endpoint_url=endpoint_url,
        # region_name=region,
        # credentials_profile_name="<profile_name>",
    )

retriever2 = AmazonKnowledgeBasesRetriever(
        knowledge_base_id=kb_id_2,
        retrieval_config={"vectorSearchConfiguration": 
                          {"numberOfResults": 10,
                           'overrideSearchType': "HYBRID", # optional
                           }
                          },
        # endpoint_url=endpoint_url,
        # region_name=region,
        # credentials_profile_name="<profile_name>",
    )

docs1 = retriever1.get_relevant_documents(
        query=query
    )

docs2 = retriever2.get_relevant_documents(
        query=query
    )

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


CPU times: user 187 ms, sys: 22.9 ms, total: 210 ms
Wall time: 1.42 s


### 두 KB (문서들)에 대해 검색된 결과를 각각 확인 합니다.

#### KB 1 검색 내용 출력

In [8]:
uri_list_1 = []
content_list_1 = []

for doc in docs1:
    uri = doc.metadata['location']['s3Location']['uri']
    content = doc.page_content

    # 리스트에 추가
    uri_list_1.append(uri)
    content_list_1.append(content)

    # 출력
    print(f"URI: {uri}")
    print(f"Content: {content[:100]}...")  # 내용의 처음 100자만 출력
    print("-" * 50)  # 구분선

# 저장된 데이터 확인
print(f"\n총 {len(uri_list_1)}개의 문서가 처리되었습니다.")
print(f"URI 리스트의 첫 번째 항목: {uri_list_1[0]}")
print(f"Content 리스트의 첫 번째 항목: {content_list_1[0][:100]}...")

URI: s3://240719-jesamkim-bucket/Civil_Eng_Contracts/Riyadh_456/Exhibit B. as received from ADA at Contract signing_OCR.pdf
Content: # FAST Special Conditions Final 9-30.13
There will be no direct enforcement rights against subcontra...
--------------------------------------------------
URI: s3://240719-jesamkim-bucket/Civil_Eng_Contracts/Riyadh_456/Exhibit C. General Conditions-int.pdf
Content: # RIYADH METRO PROJECT
## The Kingdom of Saudi Arabia
### Riyadh Metro Project
### Exhibit "C"
### G...
--------------------------------------------------
URI: s3://240719-jesamkim-bucket/Civil_Eng_Contracts/Riyadh_456/Exhibit B. as received from ADA at Contract signing_OCR.pdf
Content: # FAST Special Conditions Final 9-30.13 The Contractor shall at all times conduct all work under thi...
--------------------------------------------------
URI: s3://240719-jesamkim-bucket/Civil_Eng_Contracts/Riyadh_456/Exhibit C. General Conditions-int.pdf
Content: # EGIS-DAR RIYADH METRO PROJECT ## GENERAL COMM

#### KB 2 검색 내용 출력

In [9]:
uri_list_2 = []
content_list_2 = []

for doc in docs2:
    uri = doc.metadata['location']['s3Location']['uri']
    content = doc.page_content

    # 리스트에 추가
    uri_list_2.append(uri)
    content_list_2.append(content)

    # 출력
    print(f"URI: {uri}")
    print(f"Content: {content[:100]}...")  # 내용의 처음 100자만 출력
    print("-" * 50)  # 구분선

# 저장된 데이터 확인
print(f"\n총 {len(uri_list_2)}개의 문서가 처리되었습니다.")
print(f"URI 리스트의 첫 번째 항목: {uri_list_2[0]}")
print(f"Content 리스트의 첫 번째 항목: {content_list_2[0][:100]}...")

URI: s3://240719-jesamkim-bucket/Civil_Eng_Contracts/[0] RML7 ITB/Conditions of Contracts/Exhibit B_Special Conditions_Rev 01.pdf
Content: Here is the content formatted in Markdown:
| Contract Milestone No. | Milestone Description | Date+ ...
--------------------------------------------------
URI: s3://240719-jesamkim-bucket/Civil_Eng_Contracts/[0] RML7 ITB/Conditions of Contracts/Exhibit C_General Conditions_Rev 01.pdf
Content: # Document information
## GENERAL INFORMATION
| Author(s) | Georgios Rigas |
|-|-|
| Function | Proc...
--------------------------------------------------
URI: s3://240719-jesamkim-bucket/Civil_Eng_Contracts/[0] RML7 ITB/M-ESD-200000-0000-TDC-000002-01_ITT.pdf
Content: # Document information
## GENERAL INFORMATION
Author(s) | G. RIGAS
 - |  -
Function | Contracts Mana...
--------------------------------------------------
URI: s3://240719-jesamkim-bucket/Civil_Eng_Contracts/[0] RML7 ITB/Conditions of Contracts/M-ESD-200000-GN00-CON-000002-03 Addendum 01.pdf
Cont

### 두 KB 검색 결과에 대해서 차이점을 Claude 3 Sonnet 에게 질의

In [10]:
# Bedrock - Claude 3 Sonnet 호출 매서드 정의
def get_text_response(input_content):
    llm = ChatBedrock(
        credentials_profile_name=os.environ.get("BWB_PROFILE_NAME"),
        region_name=os.environ.get("BWB_REGION_NAME"),
        endpoint_url=os.environ.get("BWB_ENDPOINT_URL"),
        model_id="anthropic.claude-3-sonnet-20240229-v1:0",      # Claude 3 Sonnet
        #model_id = "anthropic.claude-3-5-sonnet-20240620-v1:0", # Claude 3.5 Sonnet
        model_kwargs={
            "max_tokens": 4096,
            "temperature": 0,
            "top_p": 0.0,
            "top_k": 0,
        }
    )
    return llm.predict(input_content)

In [23]:
context_1 = content_list_1
context_2 = content_list_2

# 답변 비교 프롬프트
comp_prompt = f"""
<result1>
{context_1}
</result1>

<result2>
{context_2}
</result2>

다음 <instruction></instruction> 에 따라서 답변을 생성하세요.

<instruction>
1. {kb_name_1} 내용인 <result1></result1> 및 {kb_name_2} 내용인 <result2></result2>를 비교 합니다.
2. 답변은 최대한 자세하게 기술해주세요.
3. Remove XML tags and replace it with **{kb_name_1}** and **{kb_name_2}**.
4. Answer in Korean.
</instruction>
"""

In [24]:
%%time

# 두 KB 비교 답변 출력
input_text = comp_prompt
response_content = get_text_response(input_content=input_text)
print(response_content)
print()

**2-civil-eng-01**과 **2-civil-eng-02**의 내용을 비교해보면 다음과 같은 차이점이 있습니다.

1. 문서 유형
**2-civil-eng-01**은 계약 조건과 관련된 내용인 반면, **2-civil-eng-02**는 프로젝트 일정, 마일스톤, 지불 조건 등에 대한 내용입니다.

2. 내용 구조
**2-civil-eng-01**은 계약 조항들을 열거한 형태인 반면, **2-civil-eng-02**는 표 형식으로 프로젝트 마일스톤과 지불 조건 등을 제시하고 있습니다.

3. 세부 내용
**2-civil-eng-01**은 계약 당사자 간의 권리와 의무, 하도급 조건, 비밀 유지, 지적재산권 등 계약과 관련된 법적 사항들을 다루고 있습니다. 반면 **2-civil-eng-02**는 프로젝트 관리 계획, 설계 및 시공 조직, 마일스톤 및 지불 조건 등 프로젝트 수행과 관련된 실무적인 내용들을 다루고 있습니다.

4. 문서 목적
**2-civil-eng-01**은 계약 당사자 간의 법적 구속력을 가지는 문서인 반면, **2-civil-eng-02**는 프로젝트 수행을 위한 가이드라인 또는 제안서 형식의 문서로 보입니다.

요약하면, **2-civil-eng-01**은 법적 구속력이 있는 계약 조건을 다루고 있고, **2-civil-eng-02**는 프로젝트 수행을 위한 실무적인 내용을 다루고 있는 문서라고 할 수 있습니다.

CPU times: user 168 ms, sys: 7.38 ms, total: 176 ms
Wall time: 16.3 s
